In [20]:
import pandas as pd
import numpy as np
import re
import os
import json
import tqdm
import random
import ollama

# 心境障碍
# patient_info = pd.read_excel(r"D:\work\code\psychgpt\data\患者信息-0422.xlsx")
# first_record = pd.read_excel(r"D:\work\code\psychgpt\data\首程.xlsx")
# case_record = pd.read_excel(r"D:\work\code\psychgpt\data\病程.xlsx")

# 精神分裂症
patient_info = pd.read_excel(r"D:\work\data_process\F20-F29\F20-F29基本信息.xlsx")
first_record = pd.read_excel(r"D:\work\data_process\F20-F29\F20-F29首程.xlsx")
case_record = pd.read_excel(r"D:\work\data_process\F20-F29\F20-F29病程.xlsx")

# 精分
# case_record = pd.read_excel(r'D:\work\code\psychgpt_benchmark\domain\2023主诊断精神分裂症随机100人-病程.xlsx')
# patient_info = pd.read_excel(r'D:\work\code\psychgpt_benchmark\domain\2023主诊断精神分裂症随机100人.xlsx')


In [3]:
# 数据集统计
drugs_kyy_kjsb = [
    '利培酮',
    '去甲文拉法辛',
    '阿戈美拉汀',
    '阿立哌唑',
    '阿米替林',
    '艾司西酞普兰',
    '氨磺必利',
    '安非他酮',
    '奥氮平',
    '奥沙西泮',
    '丙戊酸',
    '地西泮',
    '度洛西汀',
    '多虑平',
    '多奈哌齐',
    '奋乃静',
    '氟奋乃静',
    '氟伏沙明',
    '氟西汀',
    '氟哌啶醇',
    '伏硫西汀',
    '卡马西平',
    '拉莫三嗪',
    '鲁拉西酮',
    '氯丙嗪',
    '氯氮平',
    '氯米帕明',
    '美金刚',
    '米安色林',
    '米氮平',
    '米那普仑',
    '帕罗西汀',
    '齐拉西酮',
    '曲唑酮',
    '舍曲林',
    '舒必利',
    '喹硫平',
    '文拉法辛',
    '西酞普兰',
    '硝西泮',
    '唑吡坦',
    '地昔帕明',
    '去甲替林',
    '丙米嗪',
    '多塞平',
    '马普替林',
    '帕利哌酮',
    '咪达唑仑',
    '劳拉西泮',
    '碳酸锂',
]
pass

In [24]:
drugs_kyy_kjsb = [
    '利培酮',
    '去甲文拉法辛',
    '阿戈美拉汀',
    '阿立哌唑',
    '阿米替林',
    '艾司西酞普兰',
    '氨磺必利',
    '安非他酮',
    '奥氮平',
    '奥沙西泮',
    '丙戊酸',
    '地西泮',
    '度洛西汀',
    '多虑平',
    '多奈哌齐',
    '奋乃静',
    '氟奋乃静',
    '氟伏沙明',
    '氟西汀',
    '氟哌啶醇',
    '伏硫西汀',
    '卡马西平',
    '拉莫三嗪',
    '鲁拉西酮',
    '氯丙嗪',
    '氯氮平',
    '氯米帕明',
    '美金刚',
    '米安色林',
    '米氮平',
    '米那普仑',
    '帕罗西汀',
    '齐拉西酮',
    '曲唑酮',
    '舍曲林',
    '舒必利',
    '喹硫平',
    '文拉法辛',
    '西酞普兰',
    '硝西泮',
    '唑吡坦',
    '地昔帕明',
    '去甲替林',
    '丙米嗪',
    '多塞平',
    '马普替林',
    '帕利哌酮',
    '咪达唑仑',
    '劳拉西泮',
    '碳酸锂',
]

conversations_1 = []
conversations_2 = []
conversations_3 = []
conversations_4 = []

cases_missing_sth = []

for ipid in patient_info['IPID'].unique():
    try:
        patient = patient_info[patient_info['IPID']==ipid]
        first = first_record[first_record['SYXH']==ipid]['YZTL'].iloc[0]
        records = case_record[case_record['IPID']==ipid]
    except:
        print('Something missing on {}'.format(ipid))
        continue
    
    
    # 准备输入字段
    senior_record = pd.concat([records[records['BL_TYPE']=='主任医师首次查房记录'], 
                               records[records['BL_TYPE']=='副主任医师首次查房记录']], axis=0, ignore_index=True)
    senior_record_content = senior_record['YZTL'].to_list()
    if len(senior_record_content) == 0:
        junior_record = records[records['BL_TYPE']=='主治医师首次查房记录']
        junior_record_content = junior_record['YZTL'].to_list()
        if len(junior_record_content) == 0:
            continue
        else:
            senior_record_content = junior_record_content
    senior_record_content.sort()
    content = senior_record_content[0]
    try:
        mental_exam = re.search(r'精神检查：(.*?)\n', content, re.S).group(1).strip().split('精神检查')[1]
    except:
        mental_exam = re.search(r'精神检查：(.*?)\n', content, re.S).group(1).strip().split('精神检查')[0]
    try:
        base = '患者{}性，{}岁。\n'.format(patient['性别'].iloc[0], patient['年龄'].iloc[0])
        current_his = patient['现病史'].iloc[0] + '\n'
        brief_his = '现病史：' + re.search(r'4.临床表现：(.*?)5.既往史：', first, re.S).group(1).strip() + '\n' 
        # brief_his = '简要病史：' + re.search(r'4.临床表现：(.*?)5.既往史：', first, re.S).group(1).strip() + '\n'    
        past_his = '既往史：' + re.search(r'5.既往史：(.*?)6.家族史：', first, re.S).group(1).strip() + '\n' 
        family_his = '家族史：' + re.search(r'6.家族史：(.*?)7.查体及辅助检查：', first, re.S).group(1).strip() + '\n' 
        exam = '查体，辅助检查及精神检查：' + re.search(r'7.查体及辅助检查：(.*?)拟诊讨论：', first, re.S).group(1).strip() + mental_exam + '\n' 
        date_in = '入院日期：{}年{}月{}日'.format(str(patient['入院日期'].iloc[0])[:4], str(patient['入院日期'].iloc[0])[4:6], str(patient['入院日期'].iloc[0])[6:8])
    except:
        print('Something missing on {}'.format(ipid))
        continue
    
    # if random.random() > 0.7:
    #     input_info = base + brief_his + past_his + family_his + exam + date_in
    # else:
    #     input_info = base + current_his + past_his + family_his + exam + date_in
    
    input_info = base + current_his + past_his + family_his + exam + date_in
    
    # 准备输出字段-task1
    # try:
    #     zs = '主诉：{}\n'.format(re.search(r'主因“(.*?)”入院', first, re.S).group(1).strip())
    # except:
    #     zs = first_record[first_record['SYXH']==ipid]['ZS'].iloc[0]
    
    # summary_0 = '0.简要病史：' + brief_his + '\n'
    # summary_1 = '1.病程标准：' + re.search(r'1.病程标准：(.*?)2.症状学标准', first, re.S).group(1).strip() + '\n'
    # summary_2 = '2.症状学标准：' + re.search(r'2.症状学标准：(.*?)3.严重程度标准', first, re.S).group(1).strip() + '\n'
    # summary_3 = '3.严重程度标准：' + re.search(r'3.严重程度标准：(.*?)4.排除标准', first, re.S).group(1).strip() + '\n'
    # summary_4 = '4.排除标准：' + re.search(r'4.排除标准：(.*?)5.入院印象', first, re.S).group(1).strip() + '\n'
    # summary = '病例特点：{}'.format(summary_0 + summary_1 + summary_2 + summary_3 + summary_4)

    # output_task1 = zs + summary
    
    # 准备输出字段-task2
    # diag_primary = '主要诊断：{}\n'.format(patient['出院主要诊断'].iloc[0])
    # other_name = patient['出院其他诊断'].iloc[0]
    # other_code = patient['出院其他诊断ICD'].iloc[0]
    # other_mental_disease = ""
    # if 'F' in str(other_code):
    #     other_list = other_code.split(',')
    #     for idx, other_disease in enumerate(other_list):
    #         if 'F' in str(other_disease):
    #             other_mental_disease += other_name.split(',')[idx]
    #             other_mental_disease += ';'
    #     diag_other = '精神科共病诊断：{}\n'.format(other_mental_disease)
    # else:
    #     diag_other = '精神科共病诊断：无\n'
    
    # output_task2 = diag_primary + diag_other
    
    # primary_diag = patient['出院主要诊断'].iloc[0]
    # try:
    #     clean_analysis = re.search(r'对诊断及鉴别诊断分析：(.*?)诊疗原则及处理计划', content, re.S).group(1).strip()
    # except:
    #     print(content)
    #     continue
    # differential = f'主要诊断：{primary_diag}。' + '诊断及鉴别诊断分析：{}'.format("，".join(re.split(r'[,:，：]', clean_analysis)[1:]))
    # if '鉴别' not in differential:
    #     differential += '鉴别诊断：' + re.search(r'鉴别诊断：(.*?)诊疗计划', first, re.S).group(1).strip() + '\n' 
    # output_task2 = differential
    
    
    # 准备输出字段-task3
    # senior_record = pd.concat([records[records['BL_TYPE']=='主任医师首次查房记录'], 
    #                            records[records['BL_TYPE']=='副主任医师首次查房记录']], axis=0, ignore_index=True)
    # senior_record_content = senior_record['YZTL'].to_list()
    # if len(senior_record_content) == 0:
    #     junior_record = records[records['BL_TYPE']=='主治医师首次查房记录']
    #     junior_record_content = junior_record['YZTL'].to_list()
    #     if len(junior_record_content) == 0:
    #         continue
    #     else:
    #         senior_record_content = junior_record_content
    # senior_record_content.sort()
    # content = senior_record_content[0]
    # mental_exam = re.search(r'精神检查：(.*?)对诊断及鉴别诊断分析', content, re.S).group(1).strip()
    
    # try:
    #     output_task3 = '诊疗计划：' + re.search(r'诊疗计划：(.*?)医师签名', first, re.S).group(1).strip() + '\n'
    # except:
    #     print(first)
    #     continue
    
    
    # 准备输出字段-task4
    records = records.sort_values(by='YZTL')
    # prompt = "请从以下患者病程记录中，分别提取出客观的患者病情进展(包括患者的临床表现，查体，精神检查，辅助检查等结果)和主观的医生分析及诊疗决策（包括对患者检查结果的分析，对患者的诊断，治疗计划的制定等内容）。输出格式为：患者病情进展：xxx。医生诊疗决策：xxx。不要输出其他内容。"
    
    prompt = """
    你是一个精通大语言模型开发的精神科医生，现在要将精神科患者数据处理成结构化数据来训练大语言模型。
    请从以下患者病程记录中，精确提取并区分两部分内容：

    ###患者病情进展：包括客观记录的患者临床表现（如症状、查体结果、精神检查结果、辅助检查数据等）。
    
    ###医生诊疗决策：包括主观分析和决策内容（如对检查结果的分析、诊断结论、治疗计划的制定等）。
    
    请严格按照以下输出格式组织结果：

    患者病情进展：xxx
    医生诊疗决策：xxx
    
    注意事项：
    仅提取相关信息，不要输出与任务无关的内容或注释。
    确保信息分类准确，避免混淆客观病情与主观诊疗内容。
    """
    
    for index, record in records.iterrows():
        record_type = record['BL_TYPE']
        record_content = record['YZTL']
        
        response = ollama.chat(model='qwen2', messages=[
        {
            'role': 'user',
            'content': f"{prompt}患者病程记录：{record_content}",
        },
        ])
        orgnized_record = response['message']['content']
        
        # 提取患者病情进展字段和医生诊疗决策字段
        try:
            patient_progress = re.search(r'患者病情进展：(.*?)医生诊疗决策', orgnized_record, re.S).group(1).strip()
        except AttributeError:
            patient_progress = "未提取到患者病情进展字段"
        
        try:
            doctor_decision = re.search(r'医生诊疗决策：(.*)', orgnized_record, re.S).group(1).strip()
        except AttributeError:
            doctor_decision = "未提取到医生诊疗决策字段"
        
        
        input_task4 = f"患者病情进展：{patient_progress}。"
        output_task4 = f"医生诊疗决策：{doctor_decision}。"
    
    # for record in records.iterrows():
    #     if '日常查房记录' in record[1]['BL_TYPE']:
    #         if '目前治疗' in record[1]['YZTL']:
    #             drugs = re.search(r'目前治疗(.*?)\n', record[1]['YZTL'], re.DOTALL).group(1).strip()
    #             break
    
    # drugs = ''
    # for index, record in records.iterrows():
    #     if '日常查房记录' in record['BL_TYPE']:
    #         if '目前治疗' in record['YZTL']:
    #             drugs += re.search(r'目前治疗(.*?)\n', record['YZTL'], re.DOTALL).group(1).strip()
    #             drugs += ' '

    # output = '医嘱用药：{}'.format(drugs)
    # drugs_in_output = []
    # for drug in drugs_kyy_kjsb:
    #     if drug in output:
    #         if drug == '西酞普兰' and '艾司西酞普兰' not in output:
    #             drugs_in_output.append(drug)
    #         elif drug == '西酞普兰' and '艾司西酞普兰' in output:
    #             continue
    #         else:
    #             drugs_in_output.append(drug)
    
    # output_task4 = ', '.join(set(drugs_in_output))
    # if len(output_task4.split(',')) > 5:
    #     cases_missing_sth.append(ipid)
        
    # 创建数据集 task1
    # conversation1 = []
    # conversation1.append({'from':'IPID','value': str(ipid)})
    # conversation1.append({'from':'human','value': input_info})
    # conversation1.append({'from':'gpt','value': output_task1})
    # conversations_1.append({'conversations': conversation1})
    
    # # 创建数据集 task2
    # conversation2 = []
    # conversation2.append({'from':'IPID','value': str(ipid)})
    # conversation2.append({'from':'human','value': input_info})
    # conversation2.append({'from':'gpt','value': output_task2})
    # conversations_2.append({'conversations': conversation2})
    
    # # 创建数据集 task3
    # conversation3 = []
    # conversation3.append({'from':'IPID','value': str(ipid)})
    # conversation3.append({'from':'human','value': input_info})
    # conversation3.append({'from':'gpt','value': output_task3})
    # conversations_3.append({'conversations': conversation3})
    
    # # 创建数据集 task4
    conversation4 = []
    conversation4.append({'from':'IPID','value': str(ipid)})
    conversation4.append({'from':'human','value': input_info})
    conversation4.append({'from':'gpt','value': output_task4})
    conversations_4.append({'conversations': conversation4})

# print(len(conversations_1))
# print(len(conversations_2))
# print(len(conversations_3))
print(len(conversations_4))

print(cases_missing_sth)

# with open('./data/F2X-1223/task1-summary.jsonl', 'w') as file1:
#     for item in conversations_1:
#         # 将每个字典转换为JSON字符串并写入文件
#         json_str = json.dumps(item, ensure_ascii=False)
#         file1.write(json_str + '\n')

# with open('./data/F2X-1223/task2-diagnosis.jsonl', 'w') as file2:
#     for item in conversations_2:
#         # 将每个字典转换为JSON字符串并写入文件
#         json_str = json.dumps(item, ensure_ascii=False)
#         file2.write(json_str + '\n')

# with open('./data/F2X-1223/task3-treatment.jsonl', 'w') as file3:
#     for item in conversations_3:
#         # 将每个字典转换为JSON字符串并写入文件
#         json_str = json.dumps(item, ensure_ascii=False)
#         file3.write(json_str + '\n')

with open('./data/F2X-1223/task4-management.jsonl', 'w') as file4:
    for item in conversations_4:
        # 将每个字典转换为JSON字符串并写入文件
        json_str = json.dumps(item, ensure_ascii=False)
        file4.write(json_str + '\n')

KeyboardInterrupt: 

In [4]:
# 精分组

conversations_1 = []
conversations_2 = []
conversations_3 = []
conversations_4 = []

cases_missing_sth = []

for ipid in patient_info['住院唯一号'].unique():
    patient = patient_info[patient_info['住院唯一号']==ipid]
    records = case_record[case_record['住院唯一号']==ipid]
    first_record = records[records['类别']=='首次病程']['病程内容'].iloc[0]
    
    # 准备输入字段
    senior_record = pd.concat([records[records['类别']=='主任医师首次查房记录'], 
                               records[records['类别']=='副主任医师首次查房记录']], axis=0, ignore_index=True)
    senior_record_content = senior_record['病程内容'].to_list()
    if len(senior_record_content) == 0:
        junior_record = records[records['类别']=='主治医师首次查房记录']
        junior_record_content = junior_record['病程内容'].to_list()
        if len(junior_record_content) == 0:
            continue
        else:
            senior_record_content = junior_record_content
    senior_record_content.sort()
    content = senior_record_content[0]
    try:
        mental_exam = re.search(r'精神检查：(.*?)对诊断及鉴别诊断分析', content, re.S).group(1).strip().split('精神检查')[1]
    except:
        mental_exam = re.search(r'精神检查：(.*?)对诊断及鉴别诊断分析', content, re.S).group(1).strip().split('精神检查')[0]
    
    base = '患者{}性，{}岁。\n'.format(patient['性别'].iloc[0], patient['年龄'].iloc[0])
    brief_his = '简要病史：' + re.search(r'4.临床表现：(.*?)5.既往史：', first_record, re.S).group(1).strip() + '\n'    
    past_his = '既往史：' + re.search(r'5.既往史：(.*?)6.家族史：', first_record, re.S).group(1).strip() + '\n' 
    family_his = '家族史：' + re.search(r'6.家族史：(.*?)7.查体及辅助检查：', first_record, re.S).group(1).strip() + '\n' 
    exam = '查体，精神检查及辅助检查：' + re.search(r'7.查体及辅助检查：(.*?)拟诊讨论：', first_record, re.S).group(1).strip() + mental_exam + '\n' 
    if len(exam) < 20:
        cases_missing_sth.append(ipid)
    date_in = '入院日期：{}年{}月{}日'.format(str(patient['入院日期'].iloc[0])[:4], str(patient['入院日期'].iloc[0])[4:6], str(patient['入院日期'].iloc[0])[6:8])
    
    input_info = base + brief_his + past_his + family_his + exam + date_in
    
    # 准备输出字段-task1
    zs = '主诉：{}\n'.format(re.search(r'主因“(.*?)”入院', first_record, re.S).group(1).strip())
    summary_1 = '1.病程标准：' + re.search(r'1.病程标准：(.*?)2.症状学标准', first_record, re.S).group(1).strip() + '\n'
    summary_2 = '2.症状学标准：' + re.search(r'2.症状学标准：(.*?)3.严重程度标准', first_record, re.S).group(1).strip() + '\n'
    summary_3 = '3.严重程度标准：' + re.search(r'3.严重程度标准：(.*?)4.排除标准', first_record, re.S).group(1).strip() + '\n'
    summary_4 = '4.排除标准：' + re.search(r'4.排除标准：(.*?)5.入院印象', first_record, re.S).group(1).strip() + '\n'
    summary = '病例特点：{}'.format(summary_1 + summary_2 + summary_3 + summary_4)
    output_task1 = zs + summary
    
    # 准备输出字段-task2
    diag_primary = '主要诊断：{}\n'.format(patient['主要诊断名称'].iloc[0])
    other_name = patient['其他诊断名称'].iloc[0]
    other_code = patient['其他诊断ICD'].iloc[0]
    other_mental_disease = ""
    if 'F' in other_code:
        other_list = other_code.split(',')
        for idx, other_disease in enumerate(other_list):
            if 'F' in other_disease:
                other_mental_disease += other_name.split(',')[idx]
                other_mental_disease += ';'
        diag_other = '精神科共病诊断：{}\n'.format(other_mental_disease)
    else:
        diag_other = '精神科共病诊断：无\n'
    
    output_task2 = diag_primary + diag_other
    
    # 准备输出字段-task3
    # senior_record = pd.concat([records[records['类别']=='主任医师首次查房记录'], 
    #                            records[records['类别']=='副主任医师首次查房记录']], axis=0, ignore_index=True)
    # senior_record_content = senior_record['病程内容'].to_list()
    # if len(senior_record_content) == 0:
    #     junior_record = records[records['类别']=='主治医师首次查房记录']
    #     junior_record_content = junior_record['病程内容'].to_list()
    #     if len(junior_record_content) == 0:
    #         continue
    #     else:
    #         senior_record_content = junior_record_content
    # senior_record_content.sort()
    # content = senior_record_content[0]
    clean_analysis = re.search(r'对诊断及鉴别诊断分析：(.*?)诊疗原则及处理计划', content, re.S).group(1).strip()
    differential = '鉴别诊断分析：{}'.format(clean_analysis)
    
    output_task3 = differential
    
    # 准备输出字段-task4
    records = records.sort_values(by='病程内容')
    drugs = ''
    for index, record in records.iterrows():
        if '日常查房记录' in record['类别']:
            if '目前治疗' in record['病程内容']:
                drugs += re.search(r'目前治疗(.*?)\n', record['病程内容'], re.DOTALL).group(1).strip()
                drugs += ' '

    output = '医嘱用药：{}'.format(drugs)
    drugs_in_output = []
    for drug in drugs_kyy_kjsb:
        if drug in output:
            if drug == '西酞普兰' and '艾司西酞普兰' not in output:
                drugs_in_output.append(drug)
            elif drug == '西酞普兰' and '艾司西酞普兰' in output:
                continue
            else:
                drugs_in_output.append(drug)
    
    output_task4 = ', '.join(set(drugs_in_output))
    if len(output_task4.split(',')) > 5:
        cases_missing_sth.append(ipid)
        
    # 创建数据集 task1
    conversation1 = []
    conversation1.append({'from':'IPID','value': str(ipid)})
    conversation1.append({'from':'human','value': input_info})
    conversation1.append({'from':'gpt','value': output_task1})
    conversations_1.append({'conversations': conversation1})
    
    # 创建数据集 task2
    conversation2 = []
    conversation2.append({'from':'IPID','value': str(ipid)})
    conversation2.append({'from':'human','value': input_info})
    conversation2.append({'from':'gpt','value': output_task2})
    conversations_2.append({'conversations': conversation2})
    
    # 创建数据集 task3
    conversation3 = []
    conversation3.append({'from':'IPID','value': str(ipid)})
    conversation3.append({'from':'human','value': input_info})
    conversation3.append({'from':'gpt','value': output_task3})
    conversations_3.append({'conversations': conversation3})
    
    # 创建数据集 task4
    conversation4 = []
    conversation4.append({'from':'IPID','value': str(ipid)})
    conversation4.append({'from':'human','value': input_info})
    conversation4.append({'from':'gpt','value': output_task4})
    conversations_4.append({'conversations': conversation4})

print(len(conversations_1))
print(len(conversations_2))
print(len(conversations_3))
print(len(conversations_4))

print(cases_missing_sth)

with open('./data/jingfen/task1-F20.jsonl', 'w') as file1:
    for item in conversations_1:
        # 将每个字典转换为JSON字符串并写入文件
        json_str = json.dumps(item, ensure_ascii=False)
        file1.write(json_str + '\n')

with open('./data/jingfen/task2-F20.jsonl', 'w') as file2:
    for item in conversations_2:
        # 将每个字典转换为JSON字符串并写入文件
        json_str = json.dumps(item, ensure_ascii=False)
        file2.write(json_str + '\n')

with open('./data/jingfen/task3-F20.jsonl', 'w') as file3:
    for item in conversations_3:
        # 将每个字典转换为JSON字符串并写入文件
        json_str = json.dumps(item, ensure_ascii=False)
        file3.write(json_str + '\n')

with open('./data/jingfen/task4-F20.jsonl', 'w') as file4:
    for item in conversations_4:
        # 将每个字典转换为JSON字符串并写入文件
        json_str = json.dumps(item, ensure_ascii=False)
        file4.write(json_str + '\n')

99
99
99
99
[100201941898, 100201950521, 100201941875, 100201947059, 100201942654, 100201951733, 100201935802]


In [5]:
# 添加指令instruction
import random
import json

instructions_1 = [
    "请根据提供的患者信息提炼主诉，并依据ICD-10标准总结该患者的诊断标准，包括病程、症状学、严重程度和排除标准。",
    "根据患者的资料，提取其主诉，并按照ICD-10标准总结诊断标准，涵盖病程、症状、严重程度和排除标准。",
    "依据提供的患者信息，提取其主诉，并依照ICD-10标准总结诊断标准，包括病程标准、症状标准、严重程度标准和排除标准。",
    "请从患者信息中提取主诉，并依据ICD-10的相应标准，总结该患者的诊断标准，包括病程、症状、严重程度及排除标准。",
    "根据提供的患者资料，提取其主诉，并根据ICD-10标准总结其诊断标准，包括病程标准、症状学标准、严重程度标准及排除标准。",
    "请根据患者信息提取主诉，并按照ICD-10标准总结该患者的诊断标准，包括病程、症状、严重程度和排除标准。",
    "从患者信息中提取主诉，并根据ICD-10标准总结其诊断标准，包括病程、症状、严重程度和排除标准。",
    "根据提供的患者信息，提取主诉，并按照ICD-10标准总结诊断标准，涵盖病程、症状、严重程度及排除标准。",
    "请依据提供的患者信息，提炼主诉，并依照ICD-10标准总结其诊断标准，包括病程、症状、严重程度和排除标准。",
    "根据患者资料，提取其主诉，并根据ICD-10标准总结诊断标准，包括病程、症状、严重程度及排除标准。",
    "请从患者信息中提炼出主诉，并依据ICD-10标准总结诊断标准，包括病程、症状、严重程度及排除标准。",
    "根据提供的患者资料，提取主诉，并依据ICD-10的相应标准，总结诊断标准，包括病程标准、症状学标准、严重程度标准和排除标准。",
    "请根据患者信息提炼其主诉，并依照ICD-10标准总结诊断标准，包括病程、症状、严重程度和排除标准。",
    "从患者信息中提取主诉，并依据ICD-10标准总结其诊断标准，包括病程、症状、严重程度及排除标准。",
    "根据提供的患者资料，提取其主诉，并按照ICD-10标准总结诊断标准，涵盖病程、症状、严重程度及排除标准。",
    "请依据提供的患者信息，提取主诉，并根据ICD-10标准总结诊断标准，包括病程标准、症状学标准、严重程度标准和排除标准。",
    "根据患者资料，提取其主诉，并依照ICD-10标准总结诊断标准，包括病程、症状、严重程度及排除标准。",
    "请从患者信息中提炼主诉，并按照ICD-10标准总结诊断标准，包括病程、症状、严重程度和排除标准。",
    "根据提供的患者信息，提取其主诉，并依照ICD-10标准总结诊断标准，涵盖病程、症状、严重程度及排除标准。",
    "请根据患者资料提取主诉，并依据ICD-10标准总结其诊断标准，包括病程标准、症状学标准、严重程度标准及排除标准。",
]

instructions_2 = [
    "根据下述患者信息按照ICD-10诊断标准给出主要诊断以及共病诊断（若有）的疾病名称（精确到亚型）。仅需要给出诊断疾病名称，无需进行分析。",
    "根据以下患者信息，依据ICD-10标准提供主要诊断和共病诊断（若有）的疾病名称（精确到亚型）。只需给出诊断的疾病名称，无需分析。",
    "请根据患者信息，按照ICD-10标准给出主要诊断和共病诊断（若有）的疾病名称（具体到亚型）。仅需提供疾病名称，不需要进行分析。",
    "依据下述患者信息，根据ICD-10标准提供主要诊断及共病诊断（若有）的疾病名称（详细到亚型）。只需提供诊断的疾病名称，无需分析。",
    "根据以下患者资料，按照ICD-10标准给出主要诊断和共病诊断（若有）的具体疾病名称。只需提供疾病名称，无需分析。",
    "请依据下述患者信息，按照ICD-10标准给出主要诊断和共病诊断（如有）的具体疾病名称。只需提供诊断的名称，无需分析。",
    "根据患者资料，依照ICD-10标准给出主要诊断及共病诊断（如有）的疾病名称（详细到亚型）。仅需提供疾病名称，无需进行分析。",
    "请根据以下患者信息，按照ICD-10标准给出主要诊断和共病诊断（如有）的具体疾病名称。只需提供诊断的名称，无需分析。",
    "依据下述患者资料，按照ICD-10标准给出主要诊断及共病诊断（如有）的疾病名称（具体到亚型）。仅需提供疾病名称，不需分析。",
    "根据患者信息，依据ICD-10标准提供主要诊断和共病诊断（若有）的详细疾病名称。只需提供疾病名称，无需进行分析。",
    "请依据以下患者信息，按照ICD-10标准给出主要诊断和共病诊断（若有）的具体疾病名称。仅需提供名称，无需分析。",
    "根据以下患者资料，依据ICD-10标准提供主要诊断及共病诊断（如有）的疾病名称（精确到亚型）。只需给出诊断的名称，无需进行分析。",
    "请根据下述患者信息，依照ICD-10标准给出主要诊断及共病诊断（若有）的疾病名称（详细到亚型）。仅需提供疾病名称，无需分析。",
    "根据患者资料，按照ICD-10标准给出主要诊断和共病诊断（若有）的疾病名称（具体到亚型）。只需提供名称，无需分析。",
    "依据以下患者信息，依照ICD-10标准提供主要诊断和共病诊断（如有）的详细疾病名称。只需提供诊断的名称，不需要分析。",
    "请根据下述患者资料，依据ICD-10标准给出主要诊断及共病诊断（如有）的具体疾病名称。仅需提供名称，无需分析。",
    "根据患者信息，依照ICD-10标准提供主要诊断和共病诊断（若有）的疾病名称（具体到亚型）。只需给出名称，无需分析。",
    "请依据以下患者信息，按照ICD-10标准提供主要诊断及共病诊断（如有）的具体疾病名称。仅需提供诊断的名称，无需分析。",
    "根据下述患者资料，依据ICD-10标准给出主要诊断和共病诊断（若有）的详细疾病名称。只需提供名称，无需分析。",
    "请依据患者信息，依照ICD-10标准提供主要诊断及共病诊断（如有）的疾病名称（具体到亚型）。仅需提供名称，无需分析。",
]

instructions_3 = [
    "根据以下患者信息，进行精神心理疾病之间的临床鉴别诊断分析，给出主要诊断和可能的鉴别诊断。",
    "根据以下患者信息，进行精神心理疾病的临床鉴别诊断，确定主要诊断并列出可能的鉴别诊断。",
    "根据提供的患者资料，进行精神心理疾病的鉴别诊断分析，确定主要诊断和可能的鉴别诊断。",
    "请根据以下患者信息，进行精神心理疾病的临床分析，提供主要诊断及可能的鉴别诊断。",
    "依据以下患者资料，进行精神心理疾病的鉴别诊断分析，给出主要诊断及潜在的鉴别诊断。",
    "根据以下患者信息，分析精神心理疾病的临床表现，提供主要诊断及可能的鉴别诊断。",
    "请依据以下患者资料，进行精神心理疾病的鉴别诊断，确定主要诊断并列出可能的鉴别诊断。",
    "根据提供的患者信息，进行精神心理疾病的临床分析，给出主要诊断及潜在的鉴别诊断。",
    "请根据以下患者资料，分析精神心理疾病的临床表现，提供主要诊断及可能的鉴别诊断。",
    "依据以下患者信息，进行精神心理疾病的鉴别诊断分析，确定主要诊断和潜在的鉴别诊断。",
    "根据以下患者资料，进行精神心理疾病的临床鉴别，提供主要诊断和可能的鉴别诊断。",
    "请依据提供的患者信息，进行精神心理疾病的鉴别诊断，给出主要诊断及潜在的鉴别诊断。",
    "根据以下患者资料，分析精神心理疾病的临床表现，确定主要诊断和可能的鉴别诊断。",
    "请根据患者信息，进行精神心理疾病的临床鉴别诊断，提供主要诊断及可能的鉴别诊断。",
    "依据患者资料，进行精神心理疾病的临床分析，给出主要诊断及潜在的鉴别诊断。",
    "根据患者信息，进行精神心理疾病的鉴别诊断，确定主要诊断和可能的鉴别诊断。",
    "请根据患者资料，进行精神心理疾病的临床鉴别诊断分析，提供主要诊断及可能的鉴别诊断。",
    "依据患者信息，分析精神心理疾病的临床表现，给出主要诊断及潜在的鉴别诊断。",
    "根据患者资料，进行精神心理疾病的鉴别诊断分析，确定主要诊断及可能的鉴别诊断。",
    "请根据提供的患者资料，分析精神心理疾病，提供主要诊断和可能的鉴别诊断。",
]

instructions_4 = [
    "根据患者当前的病情信息，给出最合适的精神科药物治疗用药建议。",
    "请依据患者的现有病情信息，提供最佳的精神科药物治疗建议。",
    "根据患者目前的病情，推荐最合适的精神科药物治疗方案。",
    "请根据患者的当前病情，给出最适合的精神科药物治疗建议。",
    "依据患者的病情信息，提供最合适的精神科药物治疗建议。",
    "根据患者的现有病情，提出最佳的精神科药物治疗方案。",
    "请依据患者当前的病情信息，推荐最适合的精神科药物治疗。",
    "根据患者的目前病情，给出最合适的精神科药物治疗方案。",
    "请根据患者的现有病情信息，提供最佳的精神科药物治疗建议。",
    "依据患者当前的病情，提出最适合的精神科药物治疗建议。",
    "根据患者的病情信息，推荐最合适的精神科药物治疗方案。",
    "请依据患者现有的病情信息，给出最合适的精神科药物治疗建议。",
    "根据患者当前的病情，提供最佳的精神科药物治疗建议。",
    "请根据患者的目前病情信息，提出最适合的精神科药物治疗方案。",
    "依据患者的现有病情，给出最合适的精神科药物治疗建议。",
    "根据患者当前的病情信息，推荐最佳的精神科药物治疗方案。",
    "请依据患者的病情信息，提供最合适的精神科药物治疗方案。",
    "根据患者的现有病情，给出最佳的精神科药物治疗建议。",
    "请根据患者的目前病情信息，推荐最适合的精神科药物治疗。",
    "依据患者当前的病情信息，提出最佳的精神科药物治疗建议。",
]



# path1 = r'D:\work\code\psychgpt\sft\data\backup1\task1.jsonl'
# path2 = r'D:\work\code\psychgpt\sft\data\backup1\task2.jsonl'
# path3 = r'D:\work\code\psychgpt\sft\data\backup1\task3.jsonl'
# path4 = r'D:\work\code\psychgpt\sft\data\backup1\task4.jsonl'

path1 = r'D:\work\code\psychgpt\sft\data\jingfen\task1-F20.jsonl'
path2 = r'D:\work\code\psychgpt\sft\data\jingfen\task2-F20.jsonl'
path3 = r'D:\work\code\psychgpt\sft\data\jingfen\task3-F20.jsonl'
path4 = r'D:\work\code\psychgpt\sft\data\jingfen\task4-F20.jsonl'

path0 = r'D:\work\code\psychgpt_benchmark\benchmark_0720\task1.jsonl'
ipid_filtout = []
with open(path0, 'r') as file0:
    for line in file0:
        newline = eval(line)
        ipid_filtout.append(newline['conversations'][0]['value'])
file0.close()



data1 = []
with open(path1, 'r') as file1:
    for line in file1:
        newline = eval(line)
        data1.append(newline)
file1.close()

data2 = []
with open(path2, 'r') as file2:
    for line in file2:
        newline = eval(line)
        data2.append(newline)
file2.close()
     
data3 = []
with open(path3, 'r') as file3:
    for line in file3:
        newline = eval(line)
        data3.append(newline)
file3.close()
        
data4 = []
with open(path4, 'r') as file4:
    for line in file4:
        newline = eval(line)
        data4.append(newline)
file4.close()

exist_in_data1 = False
exist_in_data2 = False
exist_in_data3 = False
exist_in_data4 = False

selected_data1 = []
selected_data2 = []
selected_data3 = []
selected_data4 = []

cnt = 0
for item in data1:
    ipid = item['conversations'][0]['value']
    if ipid in ipid_filtout:
        continue
    
    for item2 in data2:
        if item2['conversations'][0]['value'] == ipid:
            exist_in_data2 = True
            break
    
    for item3 in data3:
        if item3['conversations'][0]['value'] == ipid:
            exist_in_data3 = True
            break
    
    for item4 in data4:
        if item4['conversations'][0]['value'] == ipid:
            exist_in_data4 = True
            break
    
    
    if exist_in_data2 == True and exist_in_data3 == True and exist_in_data4 == True:
        print(cnt)
        # current_his = re.search(r'简要病史：(.*?)(?=查体及精神检查)', item['conversations'][1]['value'], re.DOTALL).group(1).strip()
        input_info = item['conversations'][1]['value']
    

        input1 = "{}\n 患者信息：{}\n".format(random.choice(instructions_1), input_info)
        label1 = item['conversations'][2]['value']
        conversation1 = []
        conversation1.append({'from':'human','value': input1})
        conversation1.append({'from':'gpt','value': label1})
        selected_data1.append({'conversations': conversation1})
        
        input2 = "{}\n 患者信息：{}\n".format(random.choice(instructions_2), input_info)
        label2 = item2['conversations'][2]['value']
        conversation2 = []
        conversation2.append({'from':'human','value': input2})
        conversation2.append({'from':'gpt','value': label2})
        selected_data2.append({'conversations': conversation2})
        
        input3 = "{}\n 患者信息：{}\n".format(random.choice(instructions_3), input_info)
        label3 = item3['conversations'][2]['value']
        conversation3 = []
        conversation3.append({'from':'human','value': input3})
        conversation3.append({'from':'gpt','value': label3})
        selected_data3.append({'conversations': conversation3})

        input4 = "{}\n 患者信息：{}\n".format(random.choice(instructions_4), input_info)
        label4 = item4['conversations'][2]['value']
        conversation4 = []
        conversation4.append({'from':'human','value': input4})
        conversation4.append({'from':'gpt','value': label4})
        selected_data4.append({'conversations': conversation4})
        
        
        cnt += 1
        exist_in_data1 = False
        exist_in_data2 = False
        exist_in_data4 = False

with open(r'D:\work\code\psychgpt\sft\data\task1_new_F20.jsonl', 'w') as file1:
    for item in selected_data1:
        # 将每个字典转换为JSON字符串并写入文件
        json_str = json.dumps(item, ensure_ascii=False)
        file1.write(json_str + '\n')
file1.close()

with open(r'D:\work\code\psychgpt\sft\data\task2_new_F20.jsonl', 'w') as file2:
    for item in selected_data2:
        # 将每个字典转换为JSON字符串并写入文件
        json_str = json.dumps(item, ensure_ascii=False)
        file2.write(json_str + '\n')
file2.close()

with open(r'D:\work\code\psychgpt\sft\data\task3_new_F20.jsonl', 'w') as file3:
    for item in selected_data3:
        # 将每个字典转换为JSON字符串并写入文件
        json_str = json.dumps(item, ensure_ascii=False)
        file3.write(json_str + '\n')
file3.close()

with open(r'D:\work\code\psychgpt\sft\data\task4_new_F20.jsonl', 'w') as file4:
    for item in selected_data4:
        # 将每个字典转换为JSON字符串并写入文件
        json_str = json.dumps(item, ensure_ascii=False)
        file4.write(json_str + '\n')
file4.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78


In [17]:
#process task 5 with qa
import re
import json
import random

path = './data/backup/task5_w_qa.jsonl'
data = []
with open(path, 'r') as file:
    for line in file:
        newline = eval(line)
        data.append(newline)
file.close()

conversations = []
patient_infos = []
questions = []
answers = []
cnt = 0
for item in data:
    question_answer = item[1]['value']
    patient_info = item[0]['value'].replace('问题：'+item[1]['value'], "")
    question_answer = question_answer.replace('#', '').replace('*', '')
    qas = re.split('问题.*：|问题.*:|问题:|问题：|答案', question_answer)
    
    new_qas = []
    for i, qa in enumerate(qas):
        if i == 0:
            continue
        if i == len(qas)-1:
            new_qas.append(qa)
            break
        if '？' in qa or '?' in qa:
            new_qas.append(qa)
        else:
            if '？' in qas[i+1] or '?' in qas[i+1]:
                new_qas.append(qa)
            else:
                qas[i+1] = qa + qas[i+1]
                # continue
    
    if len(new_qas) == 6:
        question1 = new_qas[0]
        question2 = new_qas[2]
        question3 = new_qas[4]
        
        answer1 = new_qas[1]
        answer2 = new_qas[3]
        answer3 = new_qas[5]
    else:
        cnt += 1
        print('pass {}'.format(cnt))
        continue
    
    conversation = []
    conversation.append({'from': 'human', 'value': '{}\n 问题：{}'.format(patient_info, question1)})
    conversation.append({'from': 'gpt', 'value': answer1})
    conversations.append({'conversations': conversation})
    
    conversation = []
    conversation.append({'from': 'human', 'value': '{}\n 问题：{}'.format(patient_info, question2)})
    conversation.append({'from': 'gpt', 'value': answer2})
    conversations.append({'conversations': conversation})
    
    conversation = []
    conversation.append({'from': 'human', 'value': '{}\n 问题：{}'.format(patient_info, question3)})
    conversation.append({'from': 'gpt', 'value': answer3})
    conversations.append({'conversations': conversation})
    
random.shuffle(conversations)
with open(r'D:\work\code\psychgpt\sft\data\task5.jsonl', 'w') as file:
    for item in conversations:
        # 将每个字典转换为JSON字符串并写入文件
        json_str = json.dumps(item, ensure_ascii=False)
        file.write(json_str + '\n')
file.close()

pass 1
pass 2
pass 3
pass 4
pass 5
pass 6
pass 7
pass 8
pass 9
pass 10
pass 11
pass 12
pass 13
pass 14
pass 15
pass 16
pass 17
pass 18
pass 19
pass 20
pass 21
pass 22
pass 23
pass 24
pass 25
pass 26
pass 27
pass 28
pass 29
pass 30
pass 31
pass 32
pass 33
pass 34
pass 35
pass 36
pass 37
pass 38
pass 39
pass 40
pass 41
pass 42
pass 43
pass 44
pass 45
pass 46
pass 47
pass 48
pass 49
pass 50
pass 51
pass 52
pass 53
pass 54
pass 55
pass 56
pass 57
pass 58
pass 59
pass 60
pass 61
pass 62
pass 63
pass 64
pass 65
pass 66
pass 67
pass 68
pass 69
pass 70
pass 71
pass 72
pass 73
pass 74
pass 75
pass 76
pass 77
pass 78
pass 79
pass 80
pass 81
pass 82
pass 83
pass 84
pass 85
pass 86
pass 87
pass 88
pass 89
pass 90
pass 91
pass 92
pass 93
pass 94
pass 95
pass 96
pass 97
pass 98
pass 99
pass 100
pass 101
pass 102
pass 103
pass 104
pass 105
pass 106
pass 107
pass 108
pass 109
pass 110
pass 111
pass 112
pass 113
pass 114
pass 115
pass 116
pass 117
pass 118
pass 119
pass 120
pass 121
pass 122
pass 123
p

In [ ]:
import pandas as pd
import json
import re
import random
import os
import numpy as np

# F30-F33
path_f3x_sc = r"D:\work\data_process\F30-F33\首程.xlsx"
path_f3x_bc = r"D:\work\data_process\F30-F33\病程(2022-2024).xlsx"

# F20-F29
path_f2x_sc = r"D:\work\data_process\F20-F29\F20-F29首程.xlsx"
path_f2x_bc = r"D:\work\data_process\F20-F29\F20-F29病程.xlsx"


# Task1-summary
